<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/Project/langchain_meta_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LangSmith : https://smith.langchain.com/

In [1]:
!pip install langchain-openai
!pip install langchain
!pip install -U langchain-community
# !pip install langchainhub
!pip install -qU huggingface_hub
!pip install langchain_huggingface

In [2]:
import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "donghuna_proj"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANG_SMITH')


In [4]:
# system_prompt = """당신은 표(재무제표) 를 해석하는 금융 AI 어시스턴트 입니다.
# 당신의 임무는 주어진 테이블 형식의 재무제표를 바탕으로 흥미로운 사실을 정리하여 친절하게 답변하는 것입니다."""

# # 질의내용
# question = "대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!"

# prompt = ChatPromptTemplate.from_messages([
#     ("system", system_prompt),
#     ("user", "{input}")
# ])

# # 객체 생성
# llm = ChatOpenAI(
#     model_name="gpt-4o-mini",  # 모델명
#     api_key=userdata.get('OPEN-AI_KEY'),
#     organization=userdata.get('MY_OPENAI_ORG_ID'),
#     streaming=False,
#     temperature=0.2  # 창의성 (0.0 ~ 2.0)
# ).bind(logprobs=True)

# output_parser = StrOutputParser()

# chain = prompt | llm | output_parser
# chain.invoke(question)

# 질의
# answer = llm.stream(question)
# for token in answer:
#     print(token.content, end="", flush=True)
# response = llm.invoke(question)
# print(response.content)
# print(response.response_metadata)



스트리밍 출력

In [5]:
# print(f"ChatOpenAI: {chain.is_lc_serializable()}")

In [6]:
from huggingface_hub import login

login(token=userdata.get('HUG_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
from langchain.prompts import PromptTemplate

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
{question}<|end|>
<|assistant|>"""

prompt = PromptTemplate.from_template(template)

Huggingface Endpoint

direct 접근방법

In [8]:
# from langchain_huggingface import HuggingFaceEndpoint

# # 사용할 모델의 저장소 ID를 설정합니다.
# # repo_id = "microsoft/Phi-3-mini-4k-instruct"
# repo_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# # repo_id = "donghuna/Meta-Llama-3.1-8B-Instruct"

# llm = HuggingFaceEndpoint(
#     repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
#     max_new_tokens=256,  # 생성할 최대 토큰 길이를 설정합니다.
#     temperature=0.1,
#     huggingfacehub_api_token=userdata.get('HUG_TOKEN'),  # 허깅페이스 토큰
# )

# output_parser = StrOutputParser()

# # LLMChain을 초기화하고 프롬프트와 언어 모델을 전달합니다.
# chain = prompt | llm | StrOutputParser()
# # 질문을 전달하여 LLMChain을 실행하고 결과를 출력합니다.
# response = chain.invoke({"question": "안녕하세요? transformer 모델에 대해서 설명해주세요"})
# print(response)

In [9]:
!pip install --upgrade transformers

In [10]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_id = "donghuna/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    # model_kwargs={"torch_dtype": torch.float32},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "안녕하세요? transformer 모델에 대해서 설명해주세요"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "Yer lookin' fer a tale o' the Transformer model, eh? Alright then, settle yerself down with a pint o' grog and listen close.\n\nTransformer model be a type o' neural network that's been makin' waves in the world o' natural language processing (NLP). It's like a mighty ship, sailin' through the seas o' language, understandin' and generatin' text like a pro.\n\nHere be the key features o' the Transformer model:\n\n1. **Attention mechanism**: This be the heart o' the Transformer model. It's like a trusty compass that helps the model focus on the most important parts o' the input text. Instead o' processin' the text sequentially, like a traditional RNN, the Transformer model uses attention to look at all the words in the input text at once.\n2. **Self-attention**: This be a type o' attention mechanism that lets the model look at itself, so to speak. It's like a pirate lookin' in a mirror, makin' sure their beard be properly braided.\n3. **Encoder-Decoder a

In [11]:
messages = [
    # {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "너 뭐야 왜케 말잘해"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': '네, 제가 뭘까요? 제가 AI 인공지능 언어 모델입니다. 제 역할은 사용자가 입력한 문장을 이해하고, 의미 있는 답변을 제공하는 것입니다. 제가 말 잘하는 이유는 제가 학습한 데이터의 양과 퀄리티에 따라서입니다. 저는 수많은 텍스트 데이터를 학습하여 자연어 처리를 위한 알고리즘을 개발했습니다. 저에게 입력된 문장을 분석하고, 의미를 파악하여 적절한 답변을 제공하는 것입니다.'}


In [12]:
messages = [
    # {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "프롬프트 엔지니어링에 대해서 설명해줘. 그리고 어떤 프롬프트가 좋은 프롬프트인지 예시를 알려줘"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': '프롬프트 엔지니어링은 인공지능(AI) 모델이 사용자 입력을 이해하고 적절한 답변을 반환하는 데 중요한 역할을 하는 기술입니다. 프롬프트 엔지니어링은 인공지능 모델이 사용자 입력을 분석하고, 관련된 정보를 검색하고, 답변을 생성하는 과정을 포함합니다.\n\n프롬프트 엔지니어링의 목표는 인공지능 모델이 사용자 입력을 정확하게 이해하고, 관련된 정보를 정확하게 반환하는 것입니다. 프롬프트 엔지니어링을 잘하는 것은 인공지능 모델의 성능을 향상시키고, 사용자 경험을 개선하는 데 중요합니다.\n\n좋은 프롬프트의 특징은 다음과 같습니다.\n\n1. **명확성**: 프롬프트가 명확하고 분명해야 합니다. 사용자가 원하는 것을 정확하게 표현해야 합니다.\n2. **간결성**: 프롬프트가 간결해야 합니다. 불필요한 정보를 포함하지 않아야 합니다.\n3. **정확성**: 프롬프트가 정확해야 합니다. 사용자가 원하는 것을 정확하게 표현해야 합니다.\n4. **'}


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_id = "donghuna/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)



In [ ]:
device = 0 if torch.cuda.is_available() else -1

# 텍스트 생성 파이프라인 초기화
generator = pipeline('text-generation',
                     model=model,
                     tokenizer=tokenizer,
                     pad_token_id=tokenizer.eos_token_id,
                     device=device)

# 텍스트 생성 예제
text = "Once upon a time,"
generated_texts = generator(text)
print(generated_texts)

In [ ]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Hello?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

모델 로드 및 푸시하기